[Open in Colab](https://colab.research.google.com/)  
# AI Tools Assignment — Mastering the AI Toolkit 🛠️🧠

This notebook contains the Theory, Practical implementations, Ethics & Optimization, and the Bug-fix section for the assignment. Run cells in order. For heavy training (MNIST CNN) use Google Colab with GPU enabled (instructions in README).

In [ ]:
# Colab helper: mount Drive (optional)
try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception:
    pass

## Part 1: Theoretical Understanding (answers)

### Q1: TensorFlow vs PyTorch

- TensorFlow: static computational graph (originally), production-focused, strong tooling (TensorFlow Serving, TF Lite, TF Hub), widely used in industry. Better for deploying at scale and for ecosystems requiring tools like TFX.
- PyTorch: dynamic eager execution by default, more pythonic and easier to debug, strong research adoption. Choose PyTorch for research/experimentation and TensorFlow for some production pipelines or when specific TF tooling is required.

### Q2: Two Jupyter Notebook use cases

1. Exploratory data analysis (charts, quick data transforms, pivoting).
2. Prototyping ML models and visualizing intermediate results (training curves, confusion matrices).

### Q3: How spaCy enhances NLP vs basic string ops

spaCy provides tokenization, part-of-speech tagging, dependency parsing, pretrained NER models, and fast rule-based matchers. These produce structured linguistic annotations that are robust across texts, unlike brittle string-based rules.

### Comparative Analysis: Scikit-learn vs TensorFlow

- Target applications: Scikit-learn for classical ML (SVM, RandomForest, preprocessing), TensorFlow for deep neural networks and large-scale training.
- Ease of use: Scikit-learn often easier for beginners due to simple fit/predict API. TensorFlow (Keras) has improved usability but has more concepts.
- Community support: Both are large; TensorFlow has extensive deep-learning resources, while Scikit-learn is foundational for applied ML pipelines.

## Part 2: Practical Implementation — Scikit-learn (Iris)

In [ ]:
# Iris Classification with DecisionTree (scikit-learn)
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

# Load data
data = load_iris(as_frame=True)
X = data.data
y = data.target

# Simple preprocessing: check missing values
print('Missing values per column:
', X.isna().sum())

# Encode labels already numeric; split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train Decision Tree
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision (macro):', precision_score(y_test, y_pred, average='macro'))
print('Recall (macro):', recall_score(y_test, y_pred, average='macro'))
print('
Classification report:
', classification_report(y_test, y_pred, target_names=data.target_names))

## Part 2: Practical Implementation — Deep Learning (MNIST with TensorFlow)

In [ ]:
# MNIST CNN (TensorFlow Keras)
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
# add channel dim
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train (reduce epochs on local machine; for >95% accuracy use at least 5-10 epochs or use Colab GPU)
history = model.fit(x_train, y_train, epochs=5, batch_size=128, validation_split=0.1)

# Evaluate
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', test_acc)

# Visualize predictions on 5 samples
import random
indices = random.sample(range(len(x_test)), 5)
preds = model.predict(x_test[indices])
for i, idx in enumerate(indices):
    plt.figure(figsize=(3,3))
    plt.title(f'Pred: {np.argmax(preds[i])} | True: {y_test[idx]}')
    plt.imshow(x_test[idx].squeeze(), cmap='gray')
    plt.axis('off')
    plt.show()

# Save model for deployment
model.save('mnist_cnn.h5')

## Part 2: Practical Implementation — NLP with spaCy

In [ ]:
# spaCy NER + simple rule-based sentiment
import spacy
from spacy.matcher import PhraseMatcher

# Load small English model (ensure installed: python -m spacy download en_core_web_sm)
nlp = spacy.load('en_core_web_sm')

# Example Amazon-like reviews (for real assignment, use a dataset from Kaggle)
reviews = [
    'I love the Acme SmartWatch, the battery lasts all week and the strap is comfortable.',
    'The Zeta Vacuum is noisy and stopped working after two weeks. Terrible experience.',
    'Great headphones by SoundMax — amazing bass and clear mids.'
]

# NER extraction
for r in reviews:
    doc = nlp(r)
    ents = [(ent.text, ent.label_) for ent in doc.ents]
    print('Review:', r)
    print('Entities:', ents)
    # Simple rule-based brand/product detection using PhraseMatcher for demo

# Phrase matching for product/brand candidates
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')
patterns = [nlp.make_doc('Acme SmartWatch'), nlp.make_doc('Zeta Vacuum'), nlp.make_doc('SoundMax')]
matcher.add('PRODUCT', patterns)

for r in reviews:
    doc = nlp(r)
    matches = matcher(doc)
    found = [doc[start:end].text for _, start, end in matches]
    # Simple sentiment: count positive/negative words (toy example)
    pos_words = set(['love','great','amazing','good','excellent','comfortable'])
    neg_words = set(['terrible','noisy','bad','worst','stopped'])
    tokens = [t.text.lower() for t in doc]
    score = sum(1 for t in tokens if t in pos_words) - sum(1 for t in tokens if t in neg_words)
    sentiment = 'positive' if score>0 else ('negative' if score<0 else 'neutral')
    print('Matched products/brands:', found)
    print('Sentiment:', sentiment)
    print('---')

## Part 3: Ethics & Optimization

### Potential biases and mitigations

- MNIST: If training data lacks diversity (handwriting styles, digit styles), the model may underperform on different demographics or writing instruments. Tools: TensorFlow Fairness Indicators to check per-slice performance; augment data to include varied handwriting.
- Reviews sentiment: Rule-based sentiment may fail on sarcasm or cultural expressions. Mitigate by using labeled sentiment datasets, fine-tuning models, and auditing errors across slices.

### Bug Fix / Troubleshooting (summary)

See the provided `buggy_tensorflow_fixed.py` for a corrected TensorFlow training example and notes about common dimension mismatches and correct loss usage for classification tasks.

## End — Next steps

- Use the `requirements.txt` to create an environment.
- For MNIST training to reach >95% accuracy, open the notebook in Colab and run the MNIST CNN cell with at least 8-10 epochs and GPU runtime.
- The repository is ready for packaging and pushing to GitHub.